In [17]:
!pip install transformers sentence-transformers faiss-cpu

In [19]:


# Bibliotheken importieren
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Beispieltexte
texts = [
    "",
    "Der Vortrag wird in Berlin stattfinden",
    "Der Vortrag findet am 20 Mai 2025 statt",
    "Es wird um 19.30Uhr sein",
    "Vortrag ist freizugänglich"
    "Der Vertrag wurde am 05.01.2023 unterzeichnet",
    "Die Kündigungsfrist beträgt 6 Woche",
    "Der Kunde zahlt 14 Tage nach erhalt",
    "Die Provision beträgt 10.000€",
    "Der Standort ist in Hamburg aber es kann auch remote erfolgen",
    "Der Kunde verpflichtet sich Informationen zur genannten Stelle zu erteilen",
    "Die Deadline für Vertrag HamburgCity wird in April 2026 sein.",
    "Das Projekt BerlinCity läuft endes des Jahres 2025 aus"
]

# Embeddings erstellen
embedder = SentenceTransformer('all-MiniLM-L6-v2')
text_embeddings = embedder.encode(texts)

# FAISS-Index erstellen
dimension = text_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(text_embeddings))

# Information Retrieval
def retrieve_relevant_texts(question, top_k=2):
    question_embedding = embedder.encode([question])
    distances, indices = index.search(np.array(question_embedding), top_k)
    return [texts[i] for i in indices[0]]



In [3]:
# Generative Antwort
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def generate_answer(question, relevant_texts):
    context = " ".join(relevant_texts)
    input_text = f"Frage: {question} Kontext: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Frage stellen und Antwort generieren
question = "Was ist Retrieval-Augmented Generation?"
relevant_texts = retrieve_relevant_texts(question)
answer = generate_answer(question, relevant_texts)
print("Antwort:", answer)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Antwort: Das ist ein Beispieltext über maschinelles Lernen.


In [16]:
# Beispiel: Frage stellen und Antwort generieren
question = "Wann wird der Vertrag unterschrieben?"
relevant_texts = retrieve_relevant_texts(question)
answer = generate_answer(question, relevant_texts)
print("Antwort:", answer)

Antwort: Kontext: Der Vertrag wurde am 05.01.2023 unterzeichnet Der Vortrag wird in Berlin stattfinden.


In [20]:
question = "Wann läuft der Vertrag für Hamburg aus?"
relevant_texts = retrieve_relevant_texts(question)
answer = generate_answer(question, relevant_texts)
print("Antwort:", answer)

Antwort: Kontext: Wann läuft der Vertrag für Hamburg aus? Kontext: Die Deadline für Vertrag HamburgCity wird in April 2026 sein.
